In [1]:
import numpy as np
from define import qcode as qc
from define import globalvars as gv
from define.randchans import RandomUnitary
from define.QECCLfid.utils import Dot, Dagger, Kron
from define.QECCLfid.multi_qubit_kraus import get_process_correlated

In [2]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)

In [3]:
# Pr(synd) = sum_s,s' GS_ss' P(synd)_s'

In [4]:
def LoadChannel():
    return np.loadtxt("./../input/debug_testing/physical.txt").reshape(256, 256)
def LoadProjector():
    return np.loadtxt("./../input/debug_testing/SS.txt").reshape(64, 64)

In [5]:
def SyndromeTest(GS, P):
    # Test if any syndrome probabilities are negative.
    norm = 0
    stop = 0
    for synd in range(64):
        Pr_s = 0
        if stop == 0:
            print("P[{}, :] has {} 1s and {} -1s.".format(synd, np.count_nonzero(P[synd, :] == 1), np.count_nonzero(P[synd, :] == -1)))
        Pr_s = np.sum(np.dot(GS, P[synd, :]))/64
        norm = norm + Pr_s
        if stop == 0:
            print("Pr({}) = {}".format(synd, Pr_s))
        if Pr_s < 0:
            stop = 1
            print("Negative Pr(s) encountered.")
    print("Sum of all syndrome probabilities: {}.".format(norm))
    return None

In [6]:
# We want to construct the S x S part of the process matrix for a random independent unitary channel.
# G_ij = prod_q Tr[ U_q  (P_i)_q U^dag_q  (P_j)_q ]

In [7]:
def GetRandFullUnitaryProcess(nq):
    U = RandomUnitary(prox=0.5,dim=2**nq,method="exp")
    S = qc.GenerateGroup(qcode.S)
    GS = np.zeros((S.shape[0], S.shape[0]), dtype = np.double)
    for i in range(S.shape[0]):
        for j in range(S.shape[0]):
            Pi = Kron(*[gv.Pauli[p] for p in S[i]])
            Pj = Kron(*[gv.Pauli[p] for p in S[j]])
            contrib = np.trace(Dot(Pj,U,Pi,Dagger(U)))
            GS[i,j] = np.real(contrib)/np.power(2, nq)
    return (U,GS)

In [8]:
def GetRandUnitaryProcess(nq):
    U = np.zeros((nq, 2, 2), dtype=np.complex128)
    for q in range(nq):
        U[q, :, :] = RandomUnitary(prox=0.1, dim=2)
    S = qc.GenerateGroup(qcode.S)
    GS = np.zeros((S.shape[0], S.shape[0]), dtype = np.double)
    for i in range(S.shape[0]):
        for j in range(S.shape[0]):
            contrib = 1 + 0 * 1j
            for q in range(nq):
                contrib = contrib * np.trace(Dot(U[q, :, :], gv.Pauli[S[i, q], :, :], Dagger(U[q, :, :]), gv.Pauli[S[j, q], :, :]))
            GS[i, j] = np.real(contrib)/np.power(2, nq)
    return (U, GS)

In [9]:
# GS = LoadChannel()[:64, :64]
# P = LoadProjector()
# SyndromeTest(GS, P)

In [10]:
(U, GS) = GetRandFullUnitaryProcess(7)
P = LoadProjector()
SyndromeTest(GS, P)

P[0, :] has 64 1s and 0 -1s.
Pr(0) = 0.009714614881868238
P[1, :] has 32 1s and 32 -1s.
Pr(1) = 0.027758331971845355
P[2, :] has 32 1s and 32 -1s.
Pr(2) = 0.018953759368182714
P[3, :] has 32 1s and 32 -1s.
Pr(3) = 0.01585117731738018
P[4, :] has 32 1s and 32 -1s.
Pr(4) = 0.010322821907212164
P[5, :] has 32 1s and 32 -1s.
Pr(5) = 0.020139545397396725
P[6, :] has 32 1s and 32 -1s.
Pr(6) = 0.02281528030071301
P[7, :] has 32 1s and 32 -1s.
Pr(7) = 0.015317153430063811
P[8, :] has 32 1s and 32 -1s.
Pr(8) = 0.02098246869828929
P[9, :] has 32 1s and 32 -1s.
Pr(9) = -0.0037819091528555338
Negative Pr(s) encountered.
Negative Pr(s) encountered.
Sum of all syndrome probabilities: 0.9999999999999997.


In [13]:
kraus_dict = {0:(tuple(range(qcode.N)),[U])}

In [14]:
G_frontend = get_process_correlated(qcode, kraus_dict)

In [15]:
GS_frontend = G_frontend.reshape(256, 256)[:64, :64]

In [16]:
np.allclose(GS_frontend, GS)

True